This notebook gets the baseline joblib file for the beginning phase. Needed for the first task in the DAG (fetch_articles) to function (so that the retrain step can have embeddings to update and compare against later).

In [1]:
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split
from transformers import AutoTokenizer, AutoModel
import matplotlib.pyplot as plt
import umap
import numpy as np
from tqdm import tqdm
import random
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import NearestNeighbors
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sentence_transformers import SentenceTransformer
import joblib

/opt/anaconda3/envs/mlops/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
SHEET_CSV_URL = "https://docs.google.com/spreadsheets/d/19gFONrR0d4Ed57gGWtUqVHjjvw85WgysCh9ukhiTmBM/export?format=csv"
df = pd.read_csv(SHEET_CSV_URL)

In [3]:
df.head()

,date,title,url,publication,author,stance,topic_category,content
0,4/11/2025,Mapping Attacks on LGBTQ Rights in U.S. State ...,https://www.aclu.org/legislative-attacks-on-lg...,ACLU,None listed,Pro,1,No text content
1,4/3/25,The Human Toll of Trump's Anti-Trans Crusade,https://www.aclu.org/news/lgbtq-rights/the-hum...,ACLU,Lisa Francois,Pro,2,President Donald Trump’s assault on transgende...
2,4/3/25,Perkins et al. v. State (HB 121),https://www.aclu.org/cases/perkins-et-al-v-state,ACLU,None listed,Pro,3,"HB 121, which took effect on March 27, 2025, i..."
3,2/11/2025,Montana Senate endorses bill tying bathrooms t...,https://www.kpax.com/news/montana-senate-endor...,KPAX (local news),Jonathon Ambarian,Anti,"3, 10, 14, 17",A bill that would require transgender people i...
4,3/26/25,Library Patrons Sue Greenville County Over Wid...,https://www.aclu.org/press-releases/library-pa...,ACLU,None listed,Pro,4,"GREENVILLE, S.C. – Local library patrons, with..."


In [4]:
device = torch.device('mps' if torch.backends.mps.is_available() else 'cpu')
print('using device:', device)

using device: mps


In [5]:
# combine title and text
df['full_text'] = df['title'].fillna('') + ' ' + df['content'].fillna('')

# encode stance labels
le = LabelEncoder()
df['stance_encoded'] = le.fit_transform(df['stance'].str.lower())

print(le.classes_)

['anti' 'pro']


In [10]:
model = SentenceTransformer("all-MiniLM-L6-v2")
X = model.encode(df["full_text"].tolist(), show_progress_bar=True)
y = df["stance_encoded"].values
full_embeddings = model.encode(df['full_text'].tolist(), show_progress_bar=True)

Batches: 100%|██████████| 8/8 [00:02<00:00,  3.80it/s]


In [11]:
clf = LogisticRegression(multi_class="auto")  # or 'ovr'
clf.fit(X, y)
joblib.dump(clf, "classifier.joblib")

/opt/anaconda3/envs/mlops/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


['classifier.joblib']

In [12]:
joblib.dump(clf, "classifier.joblib")
print("✅ Saved classifier.joblib")

✅ Saved classifier.joblib
